In [1]:
# Análisis de Red de Tiendas RetailNow
# Importación de librerías

import pandas as pd
import numpy as np

In [2]:
# Carga de datos desde los archivos CSV

sales_df = pd.read_csv('sales.csv')
inventories_df = pd.read_csv('inventories.csv')
satisfaction_df = pd.read_csv('satisfaction.csv')

print("Datos cargados correctamente")
print(f"Ventas: {sales_df.shape[0]} filas")
print(f"Inventarios: {inventories_df.shape[0]} filas")
print(f"Satisfaccion: {satisfaction_df.shape[0]} filas")

Datos cargados correctamente
Ventas: 10 filas
Inventarios: 10 filas
Satisfaccion: 5 filas


In [3]:
# Limpieza de datos - eliminar filas con valores nulos

sales_df = sales_df.dropna()
inventories_df = inventories_df.dropna()
satisfaction_df = satisfaction_df.dropna()

print("\nDatos después de limpieza:")
print(f"Ventas: {sales_df.shape[0]} filas")
print(f"Inventarios: {inventories_df.shape[0]} filas")
print(f"Satisfaccion: {satisfaction_df.shape[0]} filas")


Datos después de limpieza:
Ventas: 10 filas
Inventarios: 10 filas
Satisfaccion: 5 filas


In [4]:
# Verificación de la estructura de los DataFrames

print("\n--- Estructura de Ventas ---")
print(sales_df.head())
print(sales_df.info())

print("\n--- Estructura de Inventarios ---")
print(inventories_df.head())
print(inventories_df.info())

print("\n--- Estructura de Satisfaccion ---")
print(satisfaction_df.head())
print(satisfaction_df.info())


--- Estructura de Ventas ---
   ID_Tienda    Producto  Cantidad_Vendida  Precio_Unitario Fecha_Venta
0          1  Producto A                20              100  2023-01-05
1          1  Producto B                15              200  2023-01-06
2          2  Producto A                30              100  2023-01-07
3          2  Producto C                25              300  2023-01-08
4          3  Producto A                10              100  2023-01-09
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID_Tienda         10 non-null     int64 
 1   Producto          10 non-null     object
 2   Cantidad_Vendida  10 non-null     int64 
 3   Precio_Unitario   10 non-null     int64 
 4   Fecha_Venta       10 non-null     object
dtypes: int64(3), object(2)
memory usage: 532.0+ bytes
None

--- Estructura de Inventarios ---
   ID_Tienda    Prod

In [5]:
# Exploración de datos - ventas totales por producto y tienda

# Primero calculamos los ingresos totales por cada venta
sales_df['Ingresos'] = sales_df['Cantidad_Vendida'] * sales_df['Precio_Unitario']

# Ventas totales por tienda
ventas_por_tienda = sales_df.groupby('ID_Tienda')['Ingresos'].sum().reset_index()
ventas_por_tienda.columns = ['ID_Tienda', 'Total_Ventas']

print("Ventas totales por tienda:")
print(ventas_por_tienda)

Ventas totales por tienda:
   ID_Tienda  Total_Ventas
0          1          5000
1          2         10500
2          3          9000
3          4         13000
4          5         13000


In [6]:
# Ventas totales por producto

ventas_por_producto = sales_df.groupby('Producto')['Ingresos'].sum().reset_index()
ventas_por_producto.columns = ['Producto', 'Total_Ventas']

print("\nVentas totales por producto:")
print(ventas_por_producto)


Ventas totales por producto:
     Producto  Total_Ventas
0  Producto A          8500
1  Producto B         15000
2  Producto C         27000


In [7]:
# Cantidad vendida por tienda

cantidad_por_tienda = sales_df.groupby('ID_Tienda')['Cantidad_Vendida'].sum().reset_index()
cantidad_por_tienda.columns = ['ID_Tienda', 'Total_Cantidad_Vendida']

print("\nCantidad vendida por tienda:")
print(cantidad_por_tienda)


Cantidad vendida por tienda:
   ID_Tienda  Total_Cantidad_Vendida
0          1                      35
1          2                      55
2          3                      50
3          4                      60
4          5                      50


In [8]:
# Resumen estadístico de las ventas

print("\nResumen estadístico de ventas:")
print(sales_df['Ingresos'].describe())

print("\nResumen estadístico de cantidad vendida:")
print(sales_df['Cantidad_Vendida'].describe())


Resumen estadístico de ventas:
count       10.000000
mean      5050.000000
std       3361.960407
min       1000.000000
25%       2625.000000
50%       3500.000000
75%       7875.000000
max      10500.000000
Name: Ingresos, dtype: float64

Resumen estadístico de cantidad vendida:
count    10.000000
mean     25.000000
std       9.128709
min      10.000000
25%      20.000000
50%      25.000000
75%      30.000000
max      40.000000
Name: Cantidad_Vendida, dtype: float64


In [9]:
# Análisis de inventarios - rotación de inventarios

# Primero necesitamos las ventas por tienda y producto
ventas_detalle = sales_df.groupby(['ID_Tienda', 'Producto'])['Cantidad_Vendida'].sum().reset_index()
ventas_detalle.columns = ['ID_Tienda', 'Producto', 'Cantidad_Vendida']

# Hacemos merge con inventarios
inventarios_con_ventas = inventories_df.merge(ventas_detalle, on=['ID_Tienda', 'Producto'], how='left')

# Rellenamos los NaN con 0 en caso de que no haya ventas para algún producto
inventarios_con_ventas['Cantidad_Vendida'] = inventarios_con_ventas['Cantidad_Vendida'].fillna(0)

# Calculamos la rotación de inventarios
inventarios_con_ventas['Rotacion_Inventario'] = inventarios_con_ventas['Cantidad_Vendida'] / inventarios_con_ventas['Stock_Disponible']

print("Rotación de inventarios por tienda y producto:")
print(inventarios_con_ventas[['ID_Tienda', 'Producto', 'Stock_Disponible', 'Cantidad_Vendida', 'Rotacion_Inventario']])

Rotación de inventarios por tienda y producto:
   ID_Tienda    Producto  Stock_Disponible  Cantidad_Vendida  \
0          1  Producto A                50                20   
1          1  Producto B                40                15   
2          2  Producto A                60                30   
3          2  Producto C                45                25   
4          3  Producto A                30                10   
5          3  Producto B                80                40   
6          4  Producto C                70                35   
7          4  Producto A                50                25   
8          5  Producto B                40                20   
9          5  Producto C                60                30   

   Rotacion_Inventario  
0             0.400000  
1             0.375000  
2             0.500000  
3             0.555556  
4             0.333333  
5             0.500000  
6             0.500000  
7             0.500000  
8             0.500000 

In [10]:
# Rotación de inventarios agrupada por tienda

rotacion_por_tienda = inventarios_con_ventas.groupby('ID_Tienda').agg({
    'Stock_Disponible': 'sum',
    'Cantidad_Vendida': 'sum'
}).reset_index()

rotacion_por_tienda['Rotacion_Inventario'] = rotacion_por_tienda['Cantidad_Vendida'] / rotacion_por_tienda['Stock_Disponible']

print("\nRotación de inventarios por tienda:")
print(rotacion_por_tienda)


Rotación de inventarios por tienda:
   ID_Tienda  Stock_Disponible  Cantidad_Vendida  Rotacion_Inventario
0          1                90                35             0.388889
1          2               105                55             0.523810
2          3               110                50             0.454545
3          4               120                60             0.500000
4          5               100                50             0.500000


In [11]:
# Filtrar tiendas con inventarios críticos

# Calculamos el porcentaje de productos vendidos respecto al stock
inventarios_con_ventas['Porcentaje_Vendido'] = (inventarios_con_ventas['Cantidad_Vendida'] / inventarios_con_ventas['Stock_Disponible']) * 100

# Filtramos las tiendas con menos del 10% de ventas respecto al inventario
inventarios_criticos = inventarios_con_ventas[inventarios_con_ventas['Porcentaje_Vendido'] < 10]

print("\nTiendas con inventarios críticos (menos del 10% de ventas):")
print(inventarios_criticos[['ID_Tienda', 'Producto', 'Stock_Disponible', 'Cantidad_Vendida', 'Porcentaje_Vendido']])


Tiendas con inventarios críticos (menos del 10% de ventas):
Empty DataFrame
Columns: [ID_Tienda, Producto, Stock_Disponible, Cantidad_Vendida, Porcentaje_Vendido]
Index: []


In [12]:
# Análisis de satisfacción del cliente

print("\nSatisfacción del cliente por tienda:")
print(satisfaction_df)

# Filtramos tiendas con satisfacción menor al 60%
satisfaccion_baja = satisfaction_df[satisfaction_df['Satisfacción_Promedio'] < 60]

print("\nTiendas con satisfacción baja (< 60%):")
print(satisfaccion_baja)


Satisfacción del cliente por tienda:
   ID_Tienda  Satisfacción_Promedio Fecha_Evaluación
0          1                     85       2023-01-15
1          2                     90       2023-01-15
2          3                     70       2023-01-15
3          4                     65       2023-01-15
4          5                     55       2023-01-15

Tiendas con satisfacción baja (< 60%):
   ID_Tienda  Satisfacción_Promedio Fecha_Evaluación
4          5                     55       2023-01-15


In [13]:
# Relación entre satisfacción y ventas

ventas_satisfaccion = ventas_por_tienda.merge(satisfaction_df, on='ID_Tienda', how='left')

print("\nRelación entre ventas y satisfacción:")
print(ventas_satisfaccion[['ID_Tienda', 'Total_Ventas', 'Satisfacción_Promedio']])


Relación entre ventas y satisfacción:
   ID_Tienda  Total_Ventas  Satisfacción_Promedio
0          1          5000                     85
1          2         10500                     90
2          3          9000                     70
3          4         13000                     65
4          5         13000                     55


In [14]:
# Recomendaciones para tiendas con baja satisfacción

if len(satisfaccion_baja) > 0:
    print("\nRecomendaciones para tiendas con baja satisfacción:")
    for index, row in satisfaccion_baja.iterrows():
        tienda_id = row['ID_Tienda']
        satisfaccion = row['Satisfacción_Promedio']

        ventas_tienda = ventas_por_tienda[ventas_por_tienda['ID_Tienda'] == tienda_id]['Total_Ventas'].values
        if len(ventas_tienda) > 0:
            ventas_tienda = ventas_tienda[0]
        else:
            ventas_tienda = 0

        print(f"Tienda {tienda_id}: Satisfacción {satisfaccion}% - Ventas: ${ventas_tienda}")
        print(f"  - Implementar programas de mejora en atención al cliente")
        print(f"  - Revisar calidad de productos y servicios")
        print(f"  - Capacitar al personal en servicio al cliente")
else:
    print("\nNo hay tiendas con satisfacción baja")


Recomendaciones para tiendas con baja satisfacción:
Tienda 5: Satisfacción 55% - Ventas: $13000
  - Implementar programas de mejora en atención al cliente
  - Revisar calidad de productos y servicios
  - Capacitar al personal en servicio al cliente


In [15]:
# Operaciones con Numpy - conversión de datos a arrays

# Convertimos las ventas totales a un array de Numpy
ventas_array = ventas_por_tienda['Total_Ventas'].to_numpy()

print("\nArray de ventas totales:")
print(ventas_array)


Array de ventas totales:
[ 5000 10500  9000 13000 13000]


In [16]:
# Cálculo de la mediana de las ventas totales

mediana_ventas = np.median(ventas_array)

print(f"\nMediana de las ventas totales: ${mediana_ventas}")


Mediana de las ventas totales: $10500.0


In [17]:
# Cálculo de la desviación estándar de las ventas

desviacion_std = np.std(ventas_array)

print(f"Desviación estándar de las ventas: ${desviacion_std:.2f}")

Desviación estándar de las ventas: $2973.21


In [18]:
# Simulación de proyecciones de ventas futuras

# Establecemos una semilla para reproducibilidad
np.random.seed(42)

# Número de tiendas
num_tiendas = len(ventas_array)

# Simulamos las ventas futuras para los próximos 3 meses
# Usamos la media y desviación estándar actuales como base
media_ventas = np.mean(ventas_array)

# Generamos proyecciones con una distribución normal
proyecciones_mes1 = np.random.normal(media_ventas, desviacion_std, num_tiendas)
proyecciones_mes2 = np.random.normal(media_ventas * 1.05, desviacion_std, num_tiendas)
proyecciones_mes3 = np.random.normal(media_ventas * 1.10, desviacion_std, num_tiendas)

# Nos aseguramos de que no haya valores negativos
proyecciones_mes1 = np.maximum(proyecciones_mes1, 0)
proyecciones_mes2 = np.maximum(proyecciones_mes2, 0)
proyecciones_mes3 = np.maximum(proyecciones_mes3, 0)

print("\nProyecciones de ventas futuras:")
print(f"Mes 1: {proyecciones_mes1}")
print(f"Mes 2: {proyecciones_mes2}")
print(f"Mes 3: {proyecciones_mes3}")


Proyecciones de ventas futuras:
Mes 1: [11576.83734929  9688.9106787  12025.71646685 14628.29330992
  9403.81196679]
Mes 2: [ 9908.86078034 15300.3372564  12886.74748853  9209.15230072
 12218.1469815 ]
Mes 3: [ 9732.16014399  9725.28589315 11829.40555267  5421.40887001
  5981.45058368]


In [19]:
# Estadísticas básicas sobre las proyecciones

print("\nEstadísticas de proyecciones - Mes 1:")
print(f"Media: ${np.mean(proyecciones_mes1):.2f}")
print(f"Mediana: ${np.median(proyecciones_mes1):.2f}")
print(f"Desviación estándar: ${np.std(proyecciones_mes1):.2f}")
print(f"Mínimo: ${np.min(proyecciones_mes1):.2f}")
print(f"Máximo: ${np.max(proyecciones_mes1):.2f}")

print("\nEstadísticas de proyecciones - Mes 2:")
print(f"Media: ${np.mean(proyecciones_mes2):.2f}")
print(f"Mediana: ${np.median(proyecciones_mes2):.2f}")
print(f"Desviación estándar: ${np.std(proyecciones_mes2):.2f}")
print(f"Mínimo: ${np.min(proyecciones_mes2):.2f}")
print(f"Máximo: ${np.max(proyecciones_mes2):.2f}")

print("\nEstadísticas de proyecciones - Mes 3:")
print(f"Media: ${np.mean(proyecciones_mes3):.2f}")
print(f"Mediana: ${np.median(proyecciones_mes3):.2f}")
print(f"Desviación estándar: ${np.std(proyecciones_mes3):.2f}")
print(f"Mínimo: ${np.min(proyecciones_mes3):.2f}")
print(f"Máximo: ${np.max(proyecciones_mes3):.2f}")


Estadísticas de proyecciones - Mes 1:
Media: $11464.71
Mediana: $11576.84
Desviación estándar: $1883.42
Mínimo: $9403.81
Máximo: $14628.29

Estadísticas de proyecciones - Mes 2:
Media: $11904.65
Mediana: $12218.15
Desviación estándar: $2183.68
Mínimo: $9209.15
Máximo: $15300.34

Estadísticas de proyecciones - Mes 3:
Media: $8537.94
Mediana: $9725.29
Desviación estándar: $2446.14
Mínimo: $5421.41
Máximo: $11829.41


In [20]:
# Comparación de ventas actuales vs proyecciones

print("\nComparación ventas actuales vs proyecciones:")
print(f"Ventas actuales promedio: ${media_ventas:.2f}")
print(f"Proyección promedio Mes 1: ${np.mean(proyecciones_mes1):.2f}")
print(f"Proyección promedio Mes 2: ${np.mean(proyecciones_mes2):.2f}")
print(f"Proyección promedio Mes 3: ${np.mean(proyecciones_mes3):.2f}")

crecimiento_mes1 = ((np.mean(proyecciones_mes1) - media_ventas) / media_ventas) * 100
crecimiento_mes2 = ((np.mean(proyecciones_mes2) - media_ventas) / media_ventas) * 100
crecimiento_mes3 = ((np.mean(proyecciones_mes3) - media_ventas) / media_ventas) * 100

print(f"\nCrecimiento proyectado Mes 1: {crecimiento_mes1:.2f}%")
print(f"Crecimiento proyectado Mes 2: {crecimiento_mes2:.2f}%")
print(f"Crecimiento proyectado Mes 3: {crecimiento_mes3:.2f}%")


Comparación ventas actuales vs proyecciones:
Ventas actuales promedio: $10100.00
Proyección promedio Mes 1: $11464.71
Proyección promedio Mes 2: $11904.65
Proyección promedio Mes 3: $8537.94

Crecimiento proyectado Mes 1: 13.51%
Crecimiento proyectado Mes 2: 17.87%
Crecimiento proyectado Mes 3: -15.47%


In [26]:
# Resumen final del análisis

print("\n--- RESUMEN DEL ANALISIS ---")

print(f"\nTiendas analizadas: {len(ventas_por_tienda)}")
print(f"Productos diferentes: {sales_df['Producto'].nunique()}")
print(f"Ventas totales: ${ventas_array.sum():.2f}")
print(f"Mediana de ventas: ${mediana_ventas:.2f}")
print(f"Desviación estándar: ${desviacion_std:.2f}")

print(f"\nTiendas con inventarios críticos: {len(inventarios_criticos['ID_Tienda'].unique())}")
print(f"Tiendas con baja satisfacción: {len(satisfaccion_baja)}")

if len(satisfaccion_baja) > 0:
    tiendas_atencion = [int(x) for x in satisfaccion_baja['ID_Tienda'].values]
    print(f"\nTiendas que requieren atención inmediata: {tiendas_atencion}")

print("\nProyección de crecimiento para próximos 3 meses:")
print(f"Mes 1: {crecimiento_mes1:.2f}%")
print(f"Mes 2: {crecimiento_mes2:.2f}%")
print(f"Mes 3: {crecimiento_mes3:.2f}%")


--- RESUMEN DEL ANALISIS ---

Tiendas analizadas: 5
Productos diferentes: 3
Ventas totales: $50500.00
Mediana de ventas: $10500.00
Desviación estándar: $2973.21

Tiendas con inventarios críticos: 0
Tiendas con baja satisfacción: 1

Tiendas que requieren atención inmediata: [5]

Proyección de crecimiento para próximos 3 meses:
Mes 1: 13.51%
Mes 2: 17.87%
Mes 3: -15.47%
